In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

In [ ]:
#Importazione del file HTML
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

## Environment Set-up

If running locally using jupyter, first install `segment_anything` in your environment using the [installation instructions](https://github.com/facebookresearch/segment-anything#installation) in the repository. If running from Google Colab, set `using_colab=True` below and run the cell. In Colab, be sure to select 'GPU' under 'Edit'->'Notebook Settings'->'Hardware accelerator'.

In [ ]:
using_colab = True

In [ ]:
#Ambiente

if using_colab:
    import torch
    import torchvision
    # print("PyTorch version:", torch.__version__)
    # print("Torchvision version:", torchvision.__version__)
    # print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    !pip install rasterio

    !mkdir images

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
from tensorflow.python.client import device_lib
!nvidia-smi
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import rasterio

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

## Image Processing


In [ ]:
def stretch_contrast(image):
    # Calculate the 2nd and 98th percentiles for each band
    p2 = np.percentile(image, 2, axis=(0, 1))
    p98 = np.percentile(image, 98, axis=(0, 1))

    # Stretch contrast for each band
    stretched_image = np.zeros_like(image, dtype=np.uint8)
    for i in range(image.shape[2]):
        stretched_image[:, :, i] = np.clip((image[:, :, i] - p2[i]) / (p98[i] - p2[i]) * 255, 0, 255)

    return stretched_image

In [ ]:
images_path = os.listdir(/path/to/image/collection)
path = /path/to/image/collection

images = []
for image_filename in images_path:
  image = cv2.imread(os.path.join(path,image_filename), cv2.IMREAD_UNCHANGED)
  image = stretch_contrast(image)
  images.append(image)

In [ ]:
# Reference raster of the area

path = /path/to/reference/raster
with rasterio.open(path) as src:
    raster_reference = src.read(1)

# Mask generation from seeds

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = 'cuda'

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)


In [ ]:
!pip install -qq ipympl
from google.colab import output
output.enable_custom_widget_manager()
%matplotlib ipympl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 27.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import cv2
from PIL import Image

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

### Coordinates to pixel
This allows to fix the coordinates of the seed given in input to perform a multitemporal analysis of the results

In [ ]:
def coordinates_to_pixel(lon, lat, geotransform):
    """
    Converts geographical coordinates to pixel coordinates within the image using the geotransform parameters.

    Parameters:
        lon (float): Longitude of the point.
        lat (float): Latitude of the point.
        geotransform (tuple): Geotransform parameters (affine transformation matrix).

    Returns:
        Tuple (int, int): Pixel coordinates (x, y).
    """
    # Extract geotransform parameters
    origin_x = geotransform[0]
    origin_y = geotransform[3]
    pixel_width = geotransform[1]
    pixel_height = geotransform[5]

    # Calculate pixel coordinates
    pixel_x = int((lon - origin_x) / pixel_width)
    pixel_y = int((lat - origin_y) / pixel_height)

    return pixel_x, pixel_y

In [ ]:
from osgeo import gdal

# Obtain the geotransforms of the image
raster_ds = gdal.Open(/path/to/image/)
geotransform = raster_ds.GetGeoTransform()
print("Geotransform parameters:", geotransform)

# Known coordinates
lon = 8.91738465504492
lat = 44.410415991918875

# Convert geographical coordinates to pixel coordinates
x, y = coordinates_to_pixel(lon, lat, geotransform)

print("Pixel coordinates:", x, y)

## Seed from fixed coordinates

In [ ]:
def predict_at_pixel(x, y, predictor, ax):
    global output_mask
    # Prepare input for prediction
    input_point = np.array([[x, y]])
    input_label = np.array([1])

    # Perform prediction for each predictor
    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )

    # Select the mask with highest score
    mask = masks[np.argmax(scores)]

    # Show the mask and input point
    show_mask(mask, ax, random_color=True)
    show_points(input_point, input_label, ax, marker_size=100)

    # Append the predicted mask to output_masks list
    output_mask = mask
    return output_mask

In [ ]:
# Make a prediction for each image at the same point

plt.close('all')

# Define the subplot grid
n_images = len(images)
fig, axs = plt.subplots(n_images, figsize=(20, 20))
output_masks =[]

# Loop through each image
for i, (image, ax) in enumerate(zip(images, axs)):
    ax = axs[i]
    ax.imshow(image)
    ax.axis('off')
    ax.set_title(f'Image {i}')

    # Compute the predicted mask for the current image
    predictor = SamPredictor(sam)
    predictor.set_image(image)
    output_mask = predict_at_pixel(x, y, predictor, ax)
    output_masks.append(output_mask)

    # Overlay the predicted mask on the current image
    #ax.imshow(output_mask, alpha=0.5)  # Assuming only one mask per image

plt.tight_layout()
plt.show()

In [ ]:
# Create a list containing the binary masks of the 4 periods

output_masks_bianary = []
for mask in output_masks:
  mask = mask.astype(int)
  output_masks_bianary.append(mask)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score
import numpy as np

# Compute metrics for each mask
for idx, mask in enumerate(output_masks_bianary):
    # Flatten the masks to 1D arrays
    mask_flat = mask.flatten()
    reference_mask_flat = raster_reference.flatten()

    # Compute metrics
    f1 = f1_score(reference_mask_flat, mask_flat)
    accuracy = accuracy_score(reference_mask_flat, mask_flat)
    precision = precision_score(reference_mask_flat, mask_flat)

    # Intersection over Union (IoU)
    intersection = np.logical_and(raster_reference, mask)
    union = np.logical_or(raster_reference, mask)
    iou = np.sum(intersection) / np.sum(union)

    print(f"Metrics for mask {idx}:")
    print(f"F1 Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"IoU: {iou}")
    print()

## Compute masks based on a grid of points (all over the image)

In [ ]:
def create_grid(image_width, image_height, step_size):
    # Calculate number of points in x and y directions
    num_points_x = int(np.ceil(image_width / step_size))
    num_points_y = int(np.ceil(image_height / step_size))

    # Create grid of points
    grid_points = []
    for i in range(num_points_x):
        for j in range(num_points_y):
            x = i * step_size
            y = j * step_size
            grid_points.append((x, y))

    return grid_points

In [ ]:
def predict_masks_at_grid_points(images, grid_points):
    output_masks = []

    for image in images:
        masks_per_image = []
        predictor = SamPredictor(sam)
        predictor.set_image(image)

        for point in grid_points:
            x, y = point
            input_point = np.array([[x, y]])
            input_label = np.array([1])

            # Perform prediction for the predictor
            masks, scores, logits = predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                multimask_output=True,
            )

            # Select the mask with highest score
            mask = masks[np.argmax(scores)]
            masks_per_image.append(mask)


        output_masks.append(masks_per_image)

    return output_masks

In [ ]:
grid = create_grid(images[0].shape[1], images[0].shape[0], 10)
output_masks = predict_masks_at_grid_points(images, grid)

In [ ]:
binary_output_masks = []

for masks_per_image in output_masks:
    binary_masks_per_image = []
    for mask in masks_per_image:
        binary_mask = mask.astype(int)
        binary_masks_per_image.append(binary_mask)
    binary_output_masks.append(binary_masks_per_image)

# Results

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score
import numpy as np

def compute_metrics(reference_mask_flat, mask_flat):
    # Compute metrics
    f1 = f1_score(reference_mask_flat, mask_flat)
    accuracy = accuracy_score(reference_mask_flat, mask_flat)
    precision = precision_score(reference_mask_flat, mask_flat)

    # Intersection over Union (IoU)
    intersection = np.logical_and(reference_mask_flat, mask_flat)
    union = np.logical_or(reference_mask_flat, mask_flat)
    iou = np.sum(intersection) / np.sum(union)

    return f1, accuracy, precision, iou

def find_best_mask(reference_mask_flat, masks):
    best_metrics = {'f1': 0, 'accuracy': 0, 'precision': 0, 'iou': 0}
    best_mask = None

    for idx, mask in enumerate(masks):
        mask_flat = mask.flatten()
        f1, accuracy, precision, iou = compute_metrics(reference_mask_flat, mask_flat)

        # Update best metrics if current metrics are better
        if f1 > best_metrics['f1']:
            best_metrics['f1'] = f1
            best_metrics['accuracy'] = accuracy
            best_metrics['precision'] = precision
            best_metrics['iou'] = iou
            best_mask = mask
            best_index = idx

    return best_metrics, best_mask, best_index

In [ ]:
reference_mask_flat = raster_reference.flatten()
best_metrics_per_image = []
best_masks_per_image = []
best_index_per_image = [] # index of the position in the list of the best mask -> we need it to keep track of what point coordinates we are considering

# Iterate over each image's masks
for masks_per_image in binary_output_masks:
    best_metrics, best_mask, best_index = find_best_mask(reference_mask_flat, masks_per_image)
    best_metrics_per_image.append(best_metrics)
    best_masks_per_image.append(best_mask)
    best_index_per_image.append(best_index)

In [ ]:
n_images = len(images)
fig, axs = plt.subplots(n_images, figsize=(20, 20))

# Print the best metrics for each image and plot the corresponding mask
for idx, best_metrics in enumerate(best_metrics_per_image):

    # Plotting
    ax = axs[idx]
    ax.imshow(images[idx])
    ax.axis('off')
    ax.set_title(f'Image {idx}')
    coordinates = np.array([grid[best_index_per_image[idx]]]) # Change the name of the grid
    show_mask(best_masks_per_image[idx], ax, random_color=True)
    show_points(coordinates, np.array([1]), ax, marker_size=100)


    # Printing (4 significant digits)
    print(f"Best Metrics for Image {idx}:")
    print(f"F1 Score: {best_metrics['f1']:.4f}")
    print(f"Accuracy: {best_metrics['accuracy']:.4f}")
    print(f"Precision: {best_metrics['precision']:.4f}")
    print(f"IoU: {best_metrics['iou']:.4f}")
    print()

In [ ]:
def plot_metrics_on_image(ax, image, reference, mask):
    # Compute TP, TN, FP, FN
    TP = np.logical_and(mask, reference)
    TN = np.logical_and(np.logical_not(mask), np.logical_not(reference))
    FP = np.logical_and(mask, np.logical_not(reference))
    FN = np.logical_and(np.logical_not(mask), reference)

    # Create a copy of the original image
    img_with_metrics = np.copy(image)

    # Define colors for TP, TN, FP, FN
    colors = {
        'TP': (0, 0, 255),  # Blue for True Positives
        'TN': (0, 255, 0),  # Green for True Negatives
        'FP': (255, 0, 0),  # Red for False Positives
        'FN': (255, 255, 0),  # Yellow for False Negatives
    }

    # Overlay colors for TP, TN, FP, FN on the copied image
    img_with_metrics[TP] = colors['TP']
    img_with_metrics[TN] = colors['TN']
    img_with_metrics[FP] = colors['FP']
    img_with_metrics[FN] = colors['FN']

    # Display the image
    ax.imshow(img_with_metrics)
    ax.axis('off')

In [ ]:
# Plot of the metrics

n_images = len(best_masks_per_image)
fig, axs = plt.subplots(1, n_images, figsize=(20, 5))

# Loop through each image and its corresponding mask
for idx, (mask, ax) in enumerate(zip(best_masks_per_image, axs)):
    plot_metrics_on_image(ax, images[idx], raster_reference, mask)
    ax.set_title(f'Image {idx}')  # Add title to each subplot

plt.tight_layout()
plt.show()

# 2D map of F1 score

In [ ]:
from sklearn.metrics import f1_score

f1_scores_periods = []
reference_mask_flat = raster_reference.flatten()
for mask_periods in binary_output_masks:
  f1_scores = []
  f1_scores_periods.append(f1_scores)
  for mask in mask_periods:
    mask = mask.flatten()
    f1_scores.append(f1_score(reference_mask_flat, mask))

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib.colors import LogNorm

def interpolate_f1_score_map(grid, f1_scores, image_shape):
    # Create grid points and corresponding F1 scores
    grid_x = np.array([point[0] for point in grid])
    grid_y = np.array([point[1] for point in grid])

    # Generate coordinates for the entire image
    x_new = np.arange(image_shape[1])
    y_new = np.arange(image_shape[0])
    xx, yy = np.meshgrid(x_new, y_new)

    # Interpolate F1 scores for the entire image using linear interpolation
    f1_score_map = griddata((grid_x, grid_y), f1_scores, (xx, yy), method='linear', fill_value=np.nan)

    # Use nearest interpolation to fill in the NaN values resulting from the linear interpolation
    nan_mask = np.isnan(f1_score_map)
    f1_score_map[nan_mask] = griddata((grid_x, grid_y), f1_scores, (xx, yy), method='nearest', fill_value=np.nan)[nan_mask]

    return f1_score_map



# Plot the map
def visualize_f1_score_maps(f1_score_maps):

    fig, axes = plt.subplots(1,4, figsize=(15, 5))
    axes = axes.ravel()  # Flatten the 2D array of axes to a 1D array for easy iteration

    for idx, (ax, f1_score_map) in enumerate(zip(axes, f1_score_maps)):
        im = ax.imshow(f1_score_map, cmap='viridis', interpolation='nearest')
        coordinates = np.array([grid[best_index_per_image[idx]]]) # Change the name of the grid
        show_points(coordinates, np.array([1]), ax, marker_size=100)
        ax.set_title(f'F1 Score Map {idx}')
        ax.set_xlabel('Grid Column')
        ax.set_ylabel('Grid Row')

    # Create a dedicated axis for the colorbar
    cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.5])  # [left, bottom, width, height]
    fig.colorbar(im, cax=cbar_ax, label='F1 Score')
    fig.suptitle('VV+VH+VV/VH (all filtered) - Genova harbour area', fontsize=16)

    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust layout to make space for colorbar
    plt.show()

In [ ]:
image_shape = binary_output_masks[0][0].shape
f1_maps = []
for scores in f1_scores_periods:
  f1_map = interpolate_f1_score_map(grid, scores, image_shape)
  f1_maps.append(f1_map)

In [ ]:
visualize_f1_score_maps(f1_maps)

# Comparison with AWEI+Otsu masks

In [ ]:
reference_mask_flat = raster_reference.flatten()

metrics = []
for image in images:
  f1, accuracy, precision, iou = compute_metrics(reference_mask_flat, image.flat)
  metrics_dict = {'f1': f1, 'accuracy': accuracy, 'precision': precision, 'iou': iou}
  metrics.append(metrics_dict)

In [ ]:
 # Printing (4 significant digits)
 for idx in range(4):
  metrics_image = metrics[idx]
  print(f"Metrics for Image {idx}:")
  print(f"Accuracy: {metrics_image['accuracy']:.4f}")
  print(f"Precision: {metrics_image['precision']:.4f}")
  print(f"F1 Score: {metrics_image['f1']:.4f}")
  print(f"IoU: {metrics_image['iou']:.4f}")
  print()